In [85]:
# IMPORTS

#---------------------------------------
import numpy as np

In [86]:
# COORDINATES FUNCTIONS

#----------------------------------------------------------------
# Returns the 3D vector (x, y, z) correspoding to the spherical_coordinates (r, theta, phi)
# Imput parameter is strictly a numpy array of 3 numbers: (r, theta, phi)
def spherical_to_cartesian(spherical_coordinates):
    r = spherical_coordinates[0]
    theta = spherical_coordinates[1]
    phi = spherical_coordinates[2]

    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)

    # Floating point precision would not give exact results around zeros of goniometric functions
    return np.array([x, y, z])

In [87]:
# DISTRIBUTION GENERATION FUNCTIONS - (HOMOGENEOUS SPHERE)

#---------------------------------------------------------------
# Generate a random radius within "sphere_radius" by sampling from its PDF
# This is achieved by using its inverse CDF
def generate_radius(sphere_radius, size=1):
    P = np.random.uniform(0, 1, size)

    return sphere_radius * (P ** (1 / 3))

# Generate a random inclination theta by sampling from its PDF
# This is achieved by using its inverse CDF
def generate_theta(size=1):
    P = np.random.uniform(0, 1, size)

    return np.arccos(1 - 2 * P)

# Generate a random azimuth phi; phi is uniformly distributed
def generate_phi(size=1):
    return np.random.uniform(0, 2 * np.pi, size)

# Generate a random set of coordinates of a point within "sphere_radius"
# The returned value is shaped like this: [[r_0, theta_0, phi_0],
#                                         [r_1, theta_1, phi_1],
#                                         ...
#                                         [r_(size-1), theta_(size-1), phi_(size-1)]]
def generate_sphere_point(sphere_radius, size=1):
    r = generate_radius(sphere_radius, size)
    theta = generate_theta(size)
    phi = generate_phi(size)

    # np.stack() merges the coordinates into the desired shape
    return np.stack((r, theta, phi), axis=1)

In [88]:
# PHYSICS FUNCTIONS

#-------------------------------------------------
def get_sphere_volume(radius):
    return (4 * np.pi / 3) * radius ** 3

def get_total_mass(part_mass, part_N):
    return part_mass * part_N

def get_average_density(mass, volume):
    return mass / volume

def get_collapse_time(density): # G = 1
    T_w = (3 * np.pi / density) ** (1 / 2)
    return T_w / (4 * 2 ** (1 / 2))

In [89]:
# PARAMETERS OF THE DISTRIBUTION

N = 1000 # Number of points
R = 1 # Radius of the sphere
t0 = 0 # Initial time. To be writed in the input_file
m = 1 # Mass of every any particle

# Generate the distribution
points_spherical = generate_sphere_point(R, N)

# Compute the cartesian coordinates of every particle and store it into an array
points_cartesian = np.array([spherical_to_cartesian(points_spherical[i]) for i in range(N)])

In [90]:
collapse_time = get_collapse_time(get_average_density(get_total_mass(m, N),
                                                      get_sphere_volume(R)))

print("Collapse time = " + str(collapse_time))

Collapse time = 0.035124073655203626


In [91]:
# Every particle is generated at rest
velocity_coord_str = "0 0 0"

# Converts the list of cartesian coordinates into a list of strings
# and append the mass of the particle ad the beginning
# and the initial velocity at the end
points_coord_str = ['\n' + str(m) + ' ' +
                    ' '.join(str(coord) for coord in points_cartesian[i]) + ' ' +
                    velocity_coord_str
                    for i in range(N)]

In [92]:
# Create or overwrite the input file
file_name = "input.txt"

input_file = open(file_name, 'w')

# Write a file in the proper format for the nbody_sh1 program:
#
# N
# t_0
# m_1 x_1, y_1, z_1, vx_1, vy_1, vz_1
# m_2 x_2, y_2, z_2, vx_2, vy_2, vz_2
# ...
# m_N x_N, y_N, z_N, vx_N, vy_N, vz_N
input_file.write(str(N))
input_file.write('\n' + str(t0))
input_file.writelines(points_coord_str)

input_file.close()